## Record and Save as Json

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import json

# Initialize MediaPipe BlazePose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Capture video from webcam
cap = cv2.VideoCapture(0)

# Store keypoints over time
keypoints_list = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    # Convert frame to RGB for MediaPipe
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image_rgb.flags.writeable = False

    # Run BlazePose
    results = pose.process(image_rgb)

    # Convert back to BGR for OpenCV
    image_rgb.flags.writeable = True
    image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

    # If landmarks are detected
    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        
        # Extract and store the keypoints (x, y, z) and visibility for each joint
        keypoints = []
        for landmark in landmarks:
            keypoints.append((landmark.x, landmark.y, landmark.z, landmark.visibility))
        
        keypoints_list.append(keypoints)

        # Draw the pose landmarks on the frame (for visualization)
        mp.solutions.drawing_utils.draw_landmarks(
            image_bgr, 
            results.pose_landmarks, 
            mp_pose.POSE_CONNECTIONS
        )

    # Show the frame
    cv2.imshow('Skeleton Capture', image_bgr)

    # Press q to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
# Save keypoints_list to a JSON file
with open('skeleton_data.json', 'w') as f:
    json.dump(keypoints_list, f)

cap.release()
cv2.destroyAllWindows()

In [1]:
import pandas as pd

from pose import img_keypoints_json
from pose import video_keypoints_json
from pose import img_keypoints_csv
from pose import video_keypoints_csv
from pose import play_csv
from pose import play_json
from pose import display_images
from pose import display_keypoints_imgs
from pose import cam2csv


In [3]:
cam2csv('you', 'you.csv')

C:\Users\dougl\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [ ]:
cam2json('you', 'you.json')

## Display Images

Accepts a list of image paths as the only argument, if you wanna check out a single picture than make a list with a single element which is that picture. 

example

img = ['img.jpg']

In [ ]:
import os
import glob

# Path to the 'pushups' folder
pushups_folder = 'data/pushups/images'

# Get all image paths (matching jpg, jpeg, png)
image_paths = glob.glob(os.path.join(pushups_folder, '*.[jJpP][pPnN][gG]')) + \
              glob.glob(os.path.join(pushups_folder, '*.jpeg'))

# Print or use the image paths
print(image_paths)

In [3]:
#press any key to continue
display_images(image_paths)

## Convert Pictures to CSV and JSON

In [ ]:
"""
takes a list of pictures, a label, and a filename for the output CSV file as input
"""
img_keypoints_csv(image_paths, 'pushups', 'pushups.csv')
img_keypoints_json(image_paths, 'pushups', 'pushups.json')

In [ ]:
df = pd.read_csv('pushups.csv')
df

### Visualize Keypoints in the data frame

In [6]:
#press any key to scroll through pictures
display_keypoints_imgs(df)

## Convert Key Points in Video To CSV

takes three arguments
1. a list of videos
2. a label for the video, which will be added as a column in the csv
3. a name for it to be saved as

the below example takes a video i found on youtube and converts it to a csv

In [ ]:
vid = ['data/dance/video/dance.mp4']
video_keypoints_csv(vid, 'dance', 'dance.csv')


In [ ]:
df = pd.read_csv('dance.csv')
df

## replay 

play_csv takes the csv file produced from video_keypoints_csv() and replays the video of just the skeleton and keypoints against a black frame 

In [4]:
play_csv('dance.csv')

## Convert Key Points in Video To JSON

we can basically do the same thing with json files

In [ ]:
vid = ['data/dance/video/dance.mp4']
video_keypoints_json(vid, 'dance', 'dance.json')

### Replay Json

In [2]:
play_json('dance.json')